<a href="https://colab.research.google.com/github/DrakeData/March_Madness_Bracket/blob/main/mm_bracket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import datetime
from datetime import datetime, timedelta, date
from dateutil.parser import parse
import pandas as pd
from IPython.display import clear_output

## Webscrape Data

In [2]:
main_url = 'https://www.ncaa.com/stats/basketball-men/d1/current/team/859'
page = requests.get(main_url)
soup = BeautifulSoup(page.content, 'html.parser')

print(soup.prettify())

clear_output()

In [3]:
cal_data = soup.find('tbody')
print(cal_data.prettify())

clear_output()

In [4]:
date_data = cal_data.find_all('tr')
date_data

clear_output()

In [5]:
date_data[0].find_all('td')

[<td>1</td>,
 <td><img src="https://i.turner.ncaa.com/sites/default/files/images/logos/schools/bgl/gonzaga.svg"/><a class="school" href="/schools/gonzaga">Gonzaga</a></td>,
 <td>29</td>,
 <td>930</td>,
 <td>32.07</td>]

In [6]:
date_data[1]

<tr>
<td>2</td>
<td><img src="https://i.turner.ncaa.com/sites/default/files/images/logos/schools/bgl/arizona.svg"/><a class="school" href="/schools/arizona">Arizona</a></td>
<td>34</td>
<td>1017</td>
<td>29.91</td>
</tr>

In [7]:
test = date_data[0].find_all('td')
test

[<td>1</td>,
 <td><img src="https://i.turner.ncaa.com/sites/default/files/images/logos/schools/bgl/gonzaga.svg"/><a class="school" href="/schools/gonzaga">Gonzaga</a></td>,
 <td>29</td>,
 <td>930</td>,
 <td>32.07</td>]

### loop through [NCAA website](https://www.ncaa.com/stats/basketball-men/d1/current/team/859)

In [8]:
url_ls = ['https://www.ncaa.com/stats/basketball-men/d1/current/team/859',
          'https://www.ncaa.com/stats/basketball-men/d1/current/team/859/p2',
          'https://www.ncaa.com/stats/basketball-men/d1/current/team/859/p3',
          'https://www.ncaa.com/stats/basketball-men/d1/current/team/859/p4',
          'https://www.ncaa.com/stats/basketball-men/d1/current/team/859/p5',
          'https://www.ncaa.com/stats/basketball-men/d1/current/team/859/p6',
          'https://www.ncaa.com/stats/basketball-men/d1/current/team/859/p7']

In [9]:
rank_ls = []
team_name_ls = []
dearbs_ls = []
rpg_ls = []


for url in url_ls:
  main_url = url
  page = requests.get(main_url)
  soup = BeautifulSoup(page.content, 'html.parser')

  cal_data = soup.find('tbody')
  date_data = cal_data.find_all('tr')

  for raw_data in date_data:
    main = raw_data.find_all('td')

    rank = main[0].text
    team_name = main[1].text
    dearbs = main[2].text
    rpg = main[3].text

    rank_ls.append(rank)
    team_name_ls.append(team_name)
    dearbs_ls.append(dearbs)
    rpg_ls.append(rpg)

In [10]:
main_df = pd.DataFrame({'rank': rank_ls,
                        'school_name': team_name_ls,
                        'drebs': dearbs_ls,
                        'rpg': rpg_ls})

main_df.head()

,rank,school_name,drebs,rpg
0,1,Gonzaga,29,930
1,2,Arizona,34,1017
2,3,Utah Valley,32,944
3,4,Toledo,33,973
4,5,Oral Roberts,31,907


## Create Bracket

In [11]:
main_df[main_df['school_name'] == 'Vermont']

,rank,school_name,drebs,rpg
22,23,Vermont,33,916


In [12]:
# test bracket

## Seed 1
team1 = main_df[main_df['school_name'] == 'Gonzaga']
team2 = main_df[main_df['school_name'] == 'Georgia St.']

team3 = main_df[main_df['school_name'] == 'Boise St.']
team4 = main_df[main_df['school_name'] == 'Memphis']

team5 = main_df[main_df['school_name'] == 'UConn']
team6 = main_df[main_df['school_name'] == 'New Mexico St.']

team7 = main_df[main_df['school_name'] == 'Arkansas']
team8 = main_df[main_df['school_name'] == 'Vermont']

In [13]:
main_df[main_df['school_name'] == 'Vermont'].iloc[0]

rank                23
school_name    Vermont
drebs               33
rpg                916
Name: 22, dtype: object

In [14]:
if team1.iloc[0]['drebs'] > team2.iloc[0]['drebs']:
  print(f"Predicted winner: {team1.iloc[0]['school_name']}")
else:
  print(f"Predicted winner: {team1.iloc[0]['school_name']}")

Predicted winner: Gonzaga


In [15]:
r1_dict = {'first_round_s1': [team1, team2],
             'first_round_s2': [team3, team4],
             'first_round_s3': [team5, team6],
             'first_round_s4': [team7, team8]}

In [16]:
round1_w_ls = []

for dict_key in list(r1_dict.keys()):
  if r1_dict[dict_key][0].iloc[0]['drebs'] >  r1_dict[dict_key][1].iloc[0]['drebs']:
    print(f"Predicted winner for Round 1: {r1_dict[dict_key][0].iloc[0]['school_name']}")
    round1_w_ls.append(r1_dict[dict_key][0].iloc[0]['school_name'])
  else:
    print(f"Predicted winner for Round 1: {r1_dict[dict_key][1].iloc[0]['school_name']}")
    round1_w_ls.append(r1_dict[dict_key][1].iloc[0]['school_name'])

# Create Round 2 Dictionary
r2_team_ls = []

for value in round1_w_ls:
  r2_team_ls.append(main_df[main_df['school_name'] == value])
  
  
r2_dict = {'second_round_s1': [r2_team_ls[0], r2_team_ls[1]],
           'second_round_s2': [r2_team_ls[2], r2_team_ls[3]]}

# Round 2
round2_w_ls = []

for dict_key in list(r2_dict.keys()):
  if r2_dict[dict_key][0].iloc[0]['drebs'] >  r2_dict[dict_key][1].iloc[0]['drebs']:
    print(f"Predicted winner for Round 2: {r2_dict[dict_key][0].iloc[0]['school_name']}")
    round2_w_ls.append(r2_dict[dict_key][0].iloc[0]['school_name'])
  else:
    print(f"Predicted winner for Round 2: {r2_dict[dict_key][1].iloc[0]['school_name']}")
    round2_w_ls.append(r2_dict[dict_key][1].iloc[0]['school_name'])

# Create Round 3 Dictionary
r3_team_ls = []

for value in round2_w_ls:
  r3_team_ls.append(main_df[main_df['school_name'] == value])
  
  
r3_dict = {'third_round_s1': [r2_team_ls[0], r2_team_ls[1]]}

# Round 3
round3_w_ls = []

for dict_key in list(r3_dict.keys()):
  if r3_dict[dict_key][0].iloc[0]['drebs'] >  r3_dict[dict_key][1].iloc[0]['drebs']:
    print(f"Predicted winner for Round 3: {r3_dict[dict_key][0].iloc[0]['school_name']}")
    round3_w_ls.append(r3_dict[dict_key][0].iloc[0]['school_name'])
  else:
    print(f"Predicted winner for Round 3: {r3_dict[dict_key][1].iloc[0]['school_name']}")
    round3_w_ls.append(r3_dict[dict_key][1].iloc[0]['school_name'])

Predicted winner for Round 1: Gonzaga
Predicted winner for Round 1: Boise St.
Predicted winner for Round 1: New Mexico St.
Predicted winner for Round 1: Vermont
Predicted winner for Round 2: Boise St.
Predicted winner for Round 2: Vermont
Predicted winner for Round 3: Boise St.
